In [ ]:
import sys

from PyQt5 import QtCore, QtWidgets


class DateEdit(QtWidgets.QDateEdit):
    def __init__(self, parent=None):
        super().__init__(parent, calendarPopup=True)
        self._today_button = QtWidgets.QPushButton(self.tr("Today"))
        self._today_button.clicked.connect(self._update_today)
        self.calendarWidget().layout().addWidget(self._today_button)

    #@QtCore.pyqtSlot()
    def _update_today(self):
        self._today_button.clearFocus()
        today = QtCore.QDate.currentDate()
        self.calendarWidget().setSelectedDate(today)


if __name__ == "__main__":
    app = QtWidgets.QApplication(sys.argv)
    w = DateEdit()
    w.show()
    sys.exit(app.exec_())

In [ ]:
import sys
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtCore import Qt


class TableModel(QtCore.QAbstractTableModel):
    def __init__(self, data):
        super(TableModel, self).__init__()
        self._data = data

    def data(self, index, role):
        if role == Qt.DisplayRole:
            # See below for the nested-list data structure.
            # .row() indexes into the outer list,
            # .column() indexes into the sub-list
            return self._data[index.row()][index.column()]

    def rowCount(self, index):
        # The length of the outer list.
        return len(self._data)

    def columnCount(self, index):
        # The following takes the first sub-list, and returns
        # the length (only works if all rows are an equal length)
        return len(self._data[0])


class MainWindow(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()

        self.table = QtWidgets.QTableView()

        data = [
          [4, 9, 2],
          [1, 0, 0],
          [3, 5, 0],
          [3, 3, 2],
          [7, 8, 9],
        ]

        self.model = TableModel(data)
        self.table.setModel(self.model)

        self.setCentralWidget(self.table)


app=QtWidgets.QApplication(sys.argv)
window=MainWindow()
window.show()
app.exec_()

In [ ]:
selDate = QDate.currentDate()
ist_day = selDate.addDays(1-selDate.day())
ist_day


In [ ]:
from PyQt5.QtCore import QDate, Qt

In [62]:
import os
import sys
import pandas as pd
if os.path.isfile(r'E:\source\pygame\Nado Game\pyqt5\자재관리\입고대장.xlsx'):
    LE =  [
        'E:/source/pygame/Nado Game/pyqt5/자재관리/입고대장.xlsx',
        'E:/source/pygame/Nado Game/pyqt5/자재관리/사용대장.xlsx',
        'E:/source/pygame/Nado Game/pyqt5/자재관리/동호대장.xlsx',
        ]
else: 
    LE =  [
        'C:/source/pygame/Nado Game/pyqt5/자재관리/입고대장.xlsx',
        'C:/source/pygame/Nado Game/pyqt5/자재관리/사용대장.xlsx',
        'C:/source/pygame/Nado Game/pyqt5/자재관리/동호대장.xlsx',
        ]
HEADERS = [
    ['일자', '품명', '규격', '입고수량', '구입금액', '단가', '구입업체', '비고'],
    ['일자', '동', '호', '공용', '품명' ,'규격','사용수량','비고'],
    ['품명','규격','입고수량','사용수량','재고','비고'],
    ['입출','일자','품명','규격','입고수량','누계입고','구입금액','단가','구입업체','비고'],
    ['입출','일자','공용','동','호', '품명', '규격', '사용수량', '사용누계', '비고', ],
    ['일자', '품명', '규격','입출', '입고수량','구입금액','단가','동', '호', '공용','사용수량', '재고']
]

def out_df():
    file = LE[1] #r'C:\source\pygame\Nado Game\pyqt5\자재관리\사용대장.xlsx'
    with pd.ExcelFile(file) as f:
        df = pd.read_excel(f,skiprows=0)
    return df
def in_df():
    file = LE[0] #r'C:\source\pygame\Nado Game\pyqt5\자재관리\입고대장.xlsx'
    with pd.ExcelFile(file) as f:
        df = pd.read_excel(f,skiprows=0)
    return df
def df_status_selection(df):
    from_date = '2022-06-01'
    to_date = '2022-06-26'
    items = 'LED 전구'
    spec = '11w'
    date_to_con = df['일자'] <= to_date
    date_from_con = df['일자'] >= from_date
    i_con = df['품명'] == items
    s_con = df['규격'] == spec
    if items == 'All':
        df_sel = df[(date_from_con & date_to_con)]
        return df_sel
    else:
        df_sel = df[(date_from_con & date_to_con & i_con & s_con)]
        return df_sel
def out_status_selection(df):
    from_date = '2022-02-26'
    to_date = '2022-06-26'
    items = 'LED 전구'
    spec = '11w'
    d_to_con = df['사용일'] <=to_date
    d_from_con = df['사용일'] >=from_date
    i_con = df['품명'] == items
    s_con = df['규격'] == spec
    if items == 'All':
        df_sel = df[(d_to_con & d_from_con)]
        return df_sel
    else:
        df_sel = df[(d_to_con & d_from_con & i_con & s_con)]
        return df_sel


def detailed_in_out_view():
    dfI = in_df()
    #print(dfI)
    dfI['입출'] ='입고'
    dfI.rename(columns = {'입고일':'일자'}, inplace=True)
    dfIn = df_status_selection(dfI)
    #print(dfIn)
    dfO = out_df()
    
    dfO['입출'] = '사용'
    dfO.rename(columns = {'사용일':'일자'}, inplace=True)
    dfOut = df_status_selection(dfO)
    #print(dfO)
    items_in = set(dfIn['품명'].unique())
    specs_in = set(dfIn['규격'].unique())
    items_out = set(dfOut['품명'].unique())
    specs_out = set(dfOut['규격'].unique())
    items = list(items_in.union(items_out))
    specs = list(specs_in.union(specs_out))
    temp_list = []
    for item in items:
        for spec in specs:
            dfIn_sel = dfIn[((dfIn['품명'] == item) & (dfIn['규격'] == spec))]
            dfOut_sel = dfOut[((dfOut['품명'] == item) & (dfOut['규격'] ==  spec))]
            df_on_stock = pd.merge(dfIn_sel, dfOut_sel, how = 'outer', on = ['입출','일자','품명','규격'])
            df_on_stock.fillna(0, inplace=True)
            df_on_stock.sort_values(by= '일자', inplace=True)
            #print(df_on_stock)
            df_total = df_on_stock[['일자','품명','규격','입출','입고수량','구입금액','단가','공용','동','호','사용수량']]
            df_total.reset_index(drop=True, inplace=True)
            df_total['품목누계'] = df_total['입고수량'] - df_total['사용수량']
            df_temp = df_total[['품목누계']].copy()
            df_temp_1  = df_temp.cumsum()
            df_total['품목누계'] = df_temp_1['품목누계'].copy()
            df_total[['입고수량','구입금액','단가','동','사용수량','품목누계']].astype('int')
            if df_total.empty:
                pass
            else:
                temp_list.append(df_total)
    #print(temp_list)    
    return temp_list

Total = detailed_in_out_view()
df_con = Total[0]
for part in Total[1:]:
    df_con = pd.concat([df_con, part])
    df_con.reset_index(drop=True, inplace=True)
df_con[['입고수량','구입금액','단가','동','사용수량','품목누계']] = df_con[['입고수량','구입금액','단가','동','사용수량','품목누계']].astype('int')
df_con[['입고수량','구입금액','단가','동','사용수량','품목누계']] = df_con[['입고수량','구입금액','단가','동','사용수량','품목누계']].astype('str')
df_con

<ipython-input-62-411c89fe02e6>:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total['품목누계'] = df_total['입고수량'] - df_total['사용수량']
<ipython-input-62-411c89fe02e6>:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total['품목누계'] = df_temp_1['품목누계'].copy()


,일자,품명,규격,입출,입고수량,구입금액,단가,공용,동,호,사용수량,품목누계
0,2022-06-02,LED 전구,11w,사용,0,0,0,공용,501,11층,2,-2
1,2022-06-03,LED 전구,11w,사용,0,0,0,세대,502,101,1,-3
2,2022-06-04,LED 전구,11w,입고,12,2222,185,0,0,0,0,9
3,2022-06-04,LED 전구,11w,입고,2,2222,1111,0,0,0,0,11
4,2022-06-04,LED 전구,11w,입고,1,233,233,0,0,0,0,12
5,2022-06-04,LED 전구,11w,입고,1,1,1,0,0,0,0,13
6,2022-06-04,LED 전구,11w,입고,1,1,1,0,0,0,0,14
7,2022-06-04,LED 전구,11w,입고,1,1,1,0,0,0,0,15


In [ ]:
def in_stock_view():
    list = ['LED 전구', '11w']
    on_stock_qty = items_spec_onstock(list[0], list[1])
    print(on_stock_qty)

def items_spec_onstock(items, spec):
    df_on_stock = on_stock()
    print(df_on_stock)
    print(items, spec)
    df_on_stock.set_index(['품명', '규격'], inplace=True)
    
    try:
        on_stock_qty = df_on_stock.loc[[(items, spec)],'재고'].values.tolist()
        on_stock_qty = on_stock_qty[0]
        #on_stock_qty=int(on_stock_qty)
        return on_stock_qty
    except:
        on_stock_qty = '신규항목임'
        #rint(on_stock_qty)
        return on_stock_qty
    
def on_stock():
    dfIn = in_df()
    df_in = dfIn[['품명','규격', '입고수량']].copy()
    df_in_sum = df_in.groupby(['품명','규격']).sum()

    dfOut = out_df()
    df_out = dfOut[['품명','규격', '사용수량']].copy()
    df_out_sum = df_out.groupby(['품명','규격']).sum()

    df_on_stock = pd.merge(df_in_sum, df_out_sum, how = 'outer', on = ['품명','규격'])
    df_on_stock.fillna(0, inplace=True)
    df_on_stock['일자'] = '2022-05-31'
    try:
        df_on_stock['재고'] = df_on_stock['입고수량'] - df_on_stock['사용수량']
        df_on_stock[['입고수량', '사용수량','재고']] = df_on_stock[['입고수량', '사용수량','재고']].astype('str')
        df_m = df_on_stock.reset_index().copy()
        return df_m
    except:
        pass
df = on_stock()


In [ ]:
pd.merge(df_con,df,  how = 'outer', on = ['일자','품명','규격', '입고수량','사용수량','재고'])

In [ ]:
def detailed_in_view():
    dfI = in_df()
    dfI['입출'] ='입고'
    dfI.rename(columns = {'입고일':'일자'}, inplace=True)
    dfIn = df_status_selection(dfI)
    #print(dfIn)
    
    #print(dfIn)
    items = set(dfIn['품명'].unique())
    specs = set(dfIn['규격'].unique())
    temp_list = []
    for item in items:
        #print(item)
        for spec in specs:
            #print(spec)
            #print(dfIn)
            dfIn_sel = dfIn[((dfIn['품명'] == item) & (dfIn['규격'] == spec))]
            print('dfIn_sel',dfIn_sel)
            dfIn_sel['누계입고'] = dfIn_sel['입고수량'].cumsum()
            dfIn_sel=dfIn_sel[['입출','일자','품명','규격','입고수량','누계입고','구입금액','단가','구입업체','비고']]
            temp_list.append(dfIn_sel)
    return temp_list
            
            
    

temp_list =  detailed_in_view()
temp_list[0]

In [11]:
def detailed_out_view():
    dfO = out_df()
    
    dfO['입출'] = '사용'
    dfO.rename(columns = {'사용일':'일자'}, inplace=True)
    dfOut = df_status_selection(dfO)
    print(dfOut)
    items = set(dfOut['품명'].unique())
    specs = set(dfOut['규격'].unique())
    temp_list = []
    for item in items:
        #print(item)
        for spec in specs:
            #print(spec)
            #print(dfIn)
            dfOut_sel = dfOut[((dfOut['품명'] == item) & (dfOut['규격'] ==  spec))]
            #print(dfOut_sel)
            
            dfOut_sel = dfOut[((dfOut['품명'] == item) & (dfOut['규격'] == spec))]
            #print('dfOut_sel',dfOut_sel)
            dfOut_sel['사용누계'] = dfOut_sel['사용수량'].cumsum()
            columns = dfOut_sel.columns.tolist()
            #print(columns)
            
            
            dfOut_sel = dfOut_sel[['입출','일자','공용','동','호', '품명', '규격', '사용수량', '사용누계', '비고', ]]
            temp_list.append(dfOut_sel)
    return temp_list
temp_list =  detailed_out_view()
df_total = temp_list[0]
df_total.sort_values(by=['일자'])

            일자    동    호  공용      품명   규격  사용수량      비고  입출
0   2022-05-21  501  504  공용  LED 전구  11w     1      **  사용
5   2022-05-23  501  101  공용  LED 전구  11w     2      **  사용
6   2022-05-10  501  101  공용  LED 전구  11w     2      **  사용
7   2022-05-12  501  101  공용  LED 전구  11w     2      **  사용
8   2022-05-17  501  101  공용  LED 전구  11w     2      **  사용
9   2022-05-19  501  101  공용  LED 전구  11w     2      **  사용
10  2022-05-23  501  101  공용  LED 전구  11w     2      **  사용
15  2022-05-31  501  10층  공용  LED 전구  11w     1      **  사용
16  2022-06-02  501  11층  공용  LED 전구  11w     2  계단 복도등  사용
18  2022-06-03  502  101  세대  LED 전구  11w     1     현관등  사용


,입출,일자,공용,동,호,품명,규격,사용수량,사용누계,비고
6,사용,2022-05-10,공용,501,101,LED 전구,11w,2,5,**
7,사용,2022-05-12,공용,501,101,LED 전구,11w,2,7,**
8,사용,2022-05-17,공용,501,101,LED 전구,11w,2,9,**
9,사용,2022-05-19,공용,501,101,LED 전구,11w,2,11,**
0,사용,2022-05-21,공용,501,504,LED 전구,11w,1,1,**
5,사용,2022-05-23,공용,501,101,LED 전구,11w,2,3,**
10,사용,2022-05-23,공용,501,101,LED 전구,11w,2,13,**
15,사용,2022-05-31,공용,501,10층,LED 전구,11w,1,14,**
16,사용,2022-06-02,공용,501,11층,LED 전구,11w,2,16,계단 복도등
18,사용,2022-06-03,세대,502,101,LED 전구,11w,1,17,현관등


In [74]:
def detailed_total_query():
    #self.in_out_query_combo_items_spec()
    df_list = detailed_in_out_list()
    df_in_out = detailed_in_out_df(df_list)
    df_in_out[['입고수량','구입금액','단가','동','사용수량']].astype('int')
    if len(df_in_out) == 0:
        QMessageBox.about(self, "경고", "검색 기간 동안 해당 품목에 대한 입출고 기록이 없습니다")
        return
    headers = HEADERS[5]#df_in_out.columns.values.tolist()
    df_in_out_values_list = df_in_out.values.tolist()
    return df_in_out
df_in_out = detailed_total_query()
df_in_out[['입고수량','구입금액','단가','동','사용수량']]=df_in_out[['입고수량','구입금액','단가','동','사용수량']].astype('int')
df_in_out['재고'] = df_in_out['입고수량'] - df_in_out['사용수량']

df_in_out[['입고수량','구입금액','단가','동','사용수량','재고']]=df_in_out[['입고수량','구입금액','단가','동','사용수량','재고']].astype('str')
df= on_stock_total()
df_t=pd.merge(df_in_out,df,  how = 'outer', on = ['입출','일자','품명','규격', '입고수량','사용수량','재고'])
df_t.sort_values(by='일자', inplace=True)
df_t_t = df_t['재고'].copy()
df_t_t=df_t_t.astype('int')
print(df_t_t)
tot = df_t_t.cumsum()
df_t['재고'] = tot
df_t

caller_name detailed_in_out_list
caller_name detailed_in_out_list
caller_name on_stock_total
dfIn             일자      품명   규격  입고수량    구입금액     단가     구입업체   비고
0   2022-05-03  LED 전구  11w     1       1      1  인터넷 쇼핑몰    *
70  2022-05-23  LED 전구  11w    23  542121  23570      111  111
71  2022-05-16  LED 전구  11w    23  542121  23570      111  111
72  2022-05-10  LED 전구  11w    23  542121  23570      111  111
74  2022-05-26  LED 전구  11w    45   45000   1000      kll   jj
caller_name on_stock_total
8    101
0     -2
1     -1
2     12
3      2
4      1
5      1
6      1
7      1
Name: 재고, dtype: int32


,일자,품명,규격,입출,입고수량,구입금액,단가,동,호,공용,사용수량,재고
8,2022-05-31,LED 전구,11w,재고,115,NaN,NaN,NaN,NaN,NaN,14,101
0,2022-06-02,LED 전구,11w,사용,0,0,0,501,11층,공용,2,99
1,2022-06-03,LED 전구,11w,사용,0,0,0,502,101,세대,1,98
2,2022-06-04,LED 전구,11w,입고,12,2222,185,0,0,0,0,110
3,2022-06-04,LED 전구,11w,입고,2,2222,1111,0,0,0,0,112
4,2022-06-04,LED 전구,11w,입고,1,233,233,0,0,0,0,113
5,2022-06-04,LED 전구,11w,입고,1,1,1,0,0,0,0,114
6,2022-06-04,LED 전구,11w,입고,1,1,1,0,0,0,0,115
7,2022-06-04,LED 전구,11w,입고,1,1,1,0,0,0,0,116


In [59]:
def on_stock_total():
    dfI = in_df()
    dfIn = df_status_selection(dfI)
    print('dfIn',dfIn)
    df_in = dfIn[['품명','규격', '입고수량']].copy()
    df_in_sum = df_in.groupby(['품명','규격']).sum()

    dfO = out_df()
    dfOut = df_status_selection(dfO)
    df_out = dfOut[['품명','규격', '사용수량']].copy()
    df_out_sum = df_out.groupby(['품명','규격']).sum()

    df_on_stock = pd.merge(df_in_sum, df_out_sum, how = 'outer', on = ['품명','규격'])
    df_on_stock.fillna(0, inplace=True)
    df_on_stock['일자'] = '2022-05-31'
    try:
        df_on_stock['입출'] = '재고'
        df_on_stock['재고'] = df_on_stock['입고수량'] - df_on_stock['사용수량']
        df_on_stock[['입고수량', '사용수량','재고']] = df_on_stock[['입고수량', '사용수량','재고']].astype('str')
        df_m = df_on_stock.reset_index().copy()
        return df_m
    except:
        pass


In [73]:
def callername():
    return sys._getframe(2).f_code.co_name

def df_status_selection(df):
    caller_name = sys._getframe(1).f_code.co_name
    print('caller_name',caller_name)
    from_date = '2022-06-01'
    to_date = '2022-06-06'

    items = 'LED 전구'
    spec = '11w'
    date_to_con = df['일자'] <= to_date
    date_from_con = df['일자'] >= from_date
    i_con = df['품명'] == items
    s_con = df['규격'] == spec
    if caller_name == 'on_stock_total':
        from_date = '2022-05-31'
        df_sel = df[((df['일자'] <= from_date) & i_con & s_con)]
        return df_sel
    else:
        if items == 'All':
            df_sel = df[(date_from_con & date_to_con)]
            #print(df_sel)
            return df_sel
        else:
            df_sel = df[(date_from_con & date_to_con & i_con & s_con)]
            return df_sel

In [29]:
def detailed_in_out_list():
    dfI = in_df()
    dfI['입출'] ='입고'
    dfIn = df_status_selection(dfI)
    #print(dfIn)
    dfO = out_df()
    dfO['입출'] = '사용'
    dfOut = df_status_selection(dfO)
    #print(dfOut)
    items_in = set(dfIn['품명'].unique())
    specs_in = set(dfIn['규격'].unique())
    items_out = set(dfOut['품명'].unique())
    specs_out = set(dfOut['규격'].unique())
    items = list(items_in.union(items_out))
    specs = list(specs_in.union(specs_out))
    temp_list = []
    for item in items:
        for spec in specs:
            dfIn_sel = dfIn[((dfIn['품명'] == item) & (dfIn['규격'] == spec))]
            dfOut_sel = dfOut[((dfOut['품명'] == item) & (dfOut['규격'] ==  spec))]
            df_on_stock = pd.merge(dfIn_sel, dfOut_sel, how = 'outer', on = ['입출','일자','품명','규격'])
            df_on_stock.fillna(0, inplace=True)
            df_on_stock.sort_values(by= '일자', inplace=True)
            df_total = df_on_stock[['일자','품명','규격','입출','입고수량','구입금액','단가','동','호','공용','사용수량']].copy()
            df_total.reset_index(drop=True, inplace=True)
            '''
            df_total['재고'] = df_total['입고수량'] - df_total['사용수량']
            df_temp = df_total[['재고']].copy()
            df_temp_1  = df_temp.cumsum()
            df_total['재고'] = df_temp_1['재고']
            df_total[['입고수량','구입금액','단가','동','사용수량','재고']].astype('int')
            df_total = df_total[HEADERS[5]]
            print(df_total)
            '''
            if df_total.empty:
                pass
            else:
                temp_list.append(df_total)
                
    return temp_list
d_list = detailed_in_out_list()
d_list[0]

,일자,품명,규격,입출,입고수량,구입금액,단가,동,호,공용,사용수량
0,2022-06-02,LED 전구,11w,사용,0.0,0.0,0.0,501.0,11층,공용,2.0
1,2022-06-03,LED 전구,11w,사용,0.0,0.0,0.0,502.0,101,세대,1.0
2,2022-06-04,LED 전구,11w,입고,12.0,2222.0,185.0,0.0,0,0,0.0
3,2022-06-04,LED 전구,11w,입고,2.0,2222.0,1111.0,0.0,0,0,0.0
4,2022-06-04,LED 전구,11w,입고,1.0,233.0,233.0,0.0,0,0,0.0
5,2022-06-04,LED 전구,11w,입고,1.0,1.0,1.0,0.0,0,0,0.0
6,2022-06-04,LED 전구,11w,입고,1.0,1.0,1.0,0.0,0,0,0.0
7,2022-06-04,LED 전구,11w,입고,1.0,1.0,1.0,0.0,0,0,0.0


In [20]:
def detailed_in_out_df(df_list):
    if len(df_list) == 0:
        return df_list
    df_con = df_list[0]
    for part in df_list[1:]:
        df_con = pd.concat([df_con, part])
        df_con.reset_index(drop=True, inplace=True)
    #df_con[['입고수량','구입금액','단가','동','사용수량','재고']]= df_con[['입고수량','구입금액','단가','동','사용수량','재고']].astype('int')
    #df_con[['입고수량','구입금액','단가','동','호','사용수량','재고']] = df_con[['입고수량','구입금액','단가','동','호','사용수량','재고']].astype('str')
    return df_con


In [ ]:
from IPython.display import display_html
def display_side_by_side(*args):
    """여러 데이터프레임 비교가 쉽게 옆쪽으로 표시한다"""
    html_str=''
    for df in args:
        html_str += df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'), raw=True)
dfo =out_df()
dfi =in_df()
display_side_by_side(dfi,dfo)

In [ ]:
from IPython.display import display_html
def display_side_by_side(*args):
    """여러 데이터프레임 비교가 쉽게 옆쪽으로 표시한다"""
    html_str=''
    for df in args:
        html_str += df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'), raw=True)

In [ ]:
dfo =out_df()
dfi =in_df()
dfiq = dfi.query('(품명 == "LED 전구") and (규격 == "11w")')
dfiqs = dfiq.sort_values(by=['입고일'])
#dfiqs['sum'] = dfiqs.cumsum([['입고수량']])
dfiqs['cumsum'] = dfiqs.groupby((dfiqs[[규격]]).cumsum()).입고수량.cumsum()

In [ ]:
dfo =out_df()
dfoq = dfo.query('(품명 == "LED 전구") and (규격 == "11w")')
dfoqs = dfoq.sort_values(by=['사용일'])
dfoqs

In [ ]:
#df['cumsum'] = df.groupby((df.NUM == 10).cumsum()).ENERGY.cumsum()
#출처: https://note.espriter.net/1339 [espriter:티스토리]
dfo =in_df()
dfo[['구입금액', '단가']].astype('str')
dfo.groupby(['품명', '규격','입고일']).sum(['입고수량']).groupby(level=0).cumsum().reset_index()


In [ ]:
# def cum_in_calc(self,df)
dfo =out_df()
items = dfo['품명'].unique()
#spec = dfo['규격'].unique() 
df_list = []
for i in items:
    dfo_s = dfo[(dfo['품명'] == i)]
    spec = dfo_s['규격'].unique()
    for s in spec:
        
        dfo_sel = dfo[(dfo['품명'] == i) & (dfo['규격'] == s)].copy()# & dfo['규격'] == '11w')]
        dfo_temp = dfo_sel.sort_values(by=['입고일']).copy()
        dfo_copy = dfo_temp[['입고수량']].copy()
        dfo_copy['품목 누계'] = dfo_copy.cumsum()
        dfo_temp['품목 누계']=dfo_copy['품목 누계']
        df_list.append(dfo_temp)
        
df_list[1]
df_con = df_list[0]
for l in df_list[1:]:
    df_con = pd.concat([df_con, l])
df_con
df_con.columns.tolist()
df_con[['입고일', '품명', '규격', '입고수량', '품목 누계', '구입금액', '단가', '구입업체', '비고' ]]
df_con.sort_values(by=['입고일', '품명','규격'])   
#display_side_by_side(dfo_temp, dfo_copy)

In [ ]:
def in_out_status(df):
        items = df['품명'].unique()
        df_lists = []
        for item in items:
            df_specs = df[(df['품명'] == item)]
            specs = df_specs['규격'].unique()
            for spec in specs:
                df_sel = df[(df['품명'] == item) & (df['규격'] == spec)].copy()
                try:
                    df_temp = df_sel.sort_values(by=['입고일']).copy()
                    df_copy = df_temp[['입고수량']].copy()
                except:
                    df_temp = df_sel.sort_values(by=['사용일']).copy()
                    df_copy = df_temp[['사용수량']].copy()
                df_copy['품목누계'] = df_copy.cumsum()
                df_temp['품목누계']=df_copy['품목누계']
                df_lists.append(df_temp)
        df_con = df_lists[0]
        for df_list in df_lists[1:]:
            df_con = pd.concat([df_con, df_list])
        #df_con.columns.tolist()
        try:
            df_con_1= df_con[['입고일', '품명', '규격', '입고수량', '품목누계', '구입금액', '단가', '구입업체', '비고' ]]
            df_con_1.sort_values(by=['입고일', '품명','규격'], inplace=True)
            df_con_1[['입고수량', '구입금액','단가','품목누계']] = df_con[['입고수량', '구입금액','단가','품목누계']].astype('str')
        except:
            df_con_1 = df_con[['사용일', '동','호','구분','품명', '규격', '사용수량', '품목누계', '비고' ]]
            
            df_con_1.sort_values(by=['사용일', '품명','규격'])
            df_con_1[['사용수량', '동','호']] = df_con[['사용수량', '동','호']].astype('str')

        return df_con_1
dfo =out_df()
df_out=in_out_status(dfo)
df_out

In [ ]:
dfo['구분'] = '입고'
dfo_stack=dfo.set_index(['구분','품명','규격','입고일']).stack()
dfo_stack.reset_index()
#fo_stack#['구분'] = '입고'
#dfo_stack.reset_index()


In [ ]:
from PyQt5.QtCore import QDate
qDateOutQueryStart = QDate.currentDate()
qDateOutQueryStart.addMonths(-3)
dateEdit_3.setDate(qDateOutQueryStart)